Para hacer camino Hamiltoniano: Vamos a pensar que nos pasan la matriz de adyacencia $A$ (e.g. la entrada $uv$ es 1 si y solo si hay arista de $u$ a $v$).

Este problema es bastante difícil. ¡Felicidades si lo lograste! Vamos a poner las condiciones lógicas (con implicaciones, etc.) y después las traduciremos a fórmulas de SAT.

Vamos a poner $n^2$ variables. Por cada vértice $v$ y cada entero $i\in n$, consideramos $x_{ui}$, donde $x_{ui}$ será verdadero si en el camino hamiltoniano $u$ está en la posición $i$.

Necesitamos entonces varias condiciones:

1. **En cada posición del camino sólo hay un vértice**: $x_{ui}$ y $x_{vi}$ no pueden ser verdaderas al mismo tiempo si $u\neq v$.
2. **Cada vértice debe ser visitado a lo más una vez**: $x_{ui}$ y $x_{uj}$ no pueden ser verdaderas al mismo tiempo si $i\neq j$.
3. **Todos los vértices deben ser visitados al menos una vez**: Al menos una de $x_{u0}$, $x_{u1}$, etc. debe ser verdadera.
4. **Dos vértices consecutivos deben estar conectados por una arista**: Para cada $i$, si $x_{ui}$ y $x_{v(i+1)}$, entonces $u$ y $v$ deben ser una arista. De otra manera: $A[u,v] = 0 \implies \neg x_{ui} \vee \neg x_{v(i+1)}$.

Podemos traducir a SAT de la siguiente manera (pon los cuantificadores correctos):

1. $\neg x_{ui} \vee \neg x_{vi}$
2. $\neg x_{ui} \vee \neg x_{uj}$
3. $x_{u0} \vee x_{u1} \vee \dots$
4. Cada vez que $A[u,v] == 0$ tenemos que poner $\neg x_{ui} \vee \neg x_{v(i+1)}$

Finalmente, hay que numerar las variables: $x_{00}$ es la variable 1, $x_{01}$ la 2, etc.

Vamos a hacer una función de ayuda que convierte las variables en su número:

In [1]:
def x(u,i):
    return 1 + u*n + i

In [2]:
def hamilton2sat(A):
    n = len(A)
    
    c1 = un_vertice_por_posicion(n)
    c2 = una_posicion_por_vertice(n)
    c3 = todos_aparecen(n)
    c4 = consecutivos_tienen_arista(A)
    
    clausulas = ("c un vertice por pos\n" + c1 
                + "c una pos por vert\n" + c2
                + "c todos aparecen\n" + c3
                + "c consecutivos tienen arista\n" + c4)
    
    num_clausulas = clausulas.count("0")
    
    header = f"p cnf {n} {num_clausulas}\n"
    
    return header + clausulas

In [3]:
def un_vertice_por_posicion(n):
    s = ""
    for u in range(n):
        for v in range(n):
            if u == v: continue
            for i in range(n):
                s += f"{-x(u,i)} {-x(v,i)} 0\n"
    return s

In [4]:
def una_posicion_por_vertice(n):
    s = ""
    for i in range(n):
        for j in range(n):
            if i == j: continue
            for u in range(n):
                s += f"{-x(u,i)} {-x(u,j)} 0\n"
    return s

In [5]:
def todos_aparecen(n):
    s = ""
    for u in range(n):
        s += " ".join([f"{x(u,i)}" for i in range(n)])
        s += " 0\n"
    return s

In [6]:
def consecutivos_tienen_arista(A):
    n = len(A)
    s = ""
    for u in range(n):
        for v in range(n):
            if u == v or A[u][v] == 1: continue
            for i in range(n-1):
                s += f"{-x(u,i)} {-x(v,i+1)} 0\n"
    return s

In [7]:
import random

In [8]:
def random_graph(n):
    A = [[0]*n for i in range(n)]
    for u in range(n-1):
        for v in range(u+1,n):
            if random.random() < 0.5:
                A[u][v] = 1
                A[v][u] = 1
    return A

In [9]:
n = 4
A = random_graph(n)

In [10]:
A

[[0, 1, 1, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]

In [11]:
print(hamilton2sat(A))

p cnf 4 135
c un vertice por pos
-1 -5 0
-2 -6 0
-3 -7 0
-4 -8 0
-1 -9 0
-2 -10 0
-3 -11 0
-4 -12 0
-1 -13 0
-2 -14 0
-3 -15 0
-4 -16 0
-5 -1 0
-6 -2 0
-7 -3 0
-8 -4 0
-5 -9 0
-6 -10 0
-7 -11 0
-8 -12 0
-5 -13 0
-6 -14 0
-7 -15 0
-8 -16 0
-9 -1 0
-10 -2 0
-11 -3 0
-12 -4 0
-9 -5 0
-10 -6 0
-11 -7 0
-12 -8 0
-9 -13 0
-10 -14 0
-11 -15 0
-12 -16 0
-13 -1 0
-14 -2 0
-15 -3 0
-16 -4 0
-13 -5 0
-14 -6 0
-15 -7 0
-16 -8 0
-13 -9 0
-14 -10 0
-15 -11 0
-16 -12 0
c una pos por vert
-1 -2 0
-5 -6 0
-9 -10 0
-13 -14 0
-1 -3 0
-5 -7 0
-9 -11 0
-13 -15 0
-1 -4 0
-5 -8 0
-9 -12 0
-13 -16 0
-2 -1 0
-6 -5 0
-10 -9 0
-14 -13 0
-2 -3 0
-6 -7 0
-10 -11 0
-14 -15 0
-2 -4 0
-6 -8 0
-10 -12 0
-14 -16 0
-3 -1 0
-7 -5 0
-11 -9 0
-15 -13 0
-3 -2 0
-7 -6 0
-11 -10 0
-15 -14 0
-3 -4 0
-7 -8 0
-11 -12 0
-15 -16 0
-4 -1 0
-8 -5 0
-12 -9 0
-16 -13 0
-4 -2 0
-8 -6 0
-12 -10 0
-16 -14 0
-4 -3 0
-8 -7 0
-12 -11 0
-16 -15 0
c todos aparecen
1 2 3 4 0
5 6 7 8 0
9 10 11 12 0
13 14 15 16 0
c consecutivos tienen arista
-5 

Para

```
A = [[0, 1, 1, 0], [1, 0, 1, 1], [1, 1, 0, 0], [0, 1, 0, 0]]
```

nos da de resultado

```
v -1 2 -3 -4 -5 -6 7 -8 9 -10 -11 -12 -13 -14 -15 16 0
```

Ahora vamos a interpretar eso.

In [12]:
resStr = "-1 2 -3 -4 -5 -6 7 -8 9 -10 -11 -12 -13 -14 -15 16 0"
res = [int(s) for s in resStr.split(" ")]
print("En el camino Hamiltoniano de la grafica A:")
L=[]
for i in range(n):
    for u in range(n):
        if x(u,i) in res:
            print(f"el vertice", u, "ocupa la posicion", i)
            L.append(u)
print()
for u in L:
    print(u,"--> ", end="")

En el camino Hamiltoniano de la grafica A:
el vertice 2 ocupa la posicion 0
el vertice 0 ocupa la posicion 1
el vertice 1 ocupa la posicion 2
el vertice 3 ocupa la posicion 3

2 --> 0 --> 1 --> 3 --> 